<a href="https://colab.research.google.com/github/PrishaTHE-PRO/C-140/blob/main/P_140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install kaggle

In [16]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"prisha1","key":"b0872ca4adb763a06f4d861c7127b1e3"}'}

In [17]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [18]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop

articles-sharing-reading-from-cit-deskdrop.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

In [19]:
!unzip articles-sharing-reading-from-cit-deskdrop.zip

Archive:  articles-sharing-reading-from-cit-deskdrop.zip
replace shared_articles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: shared_articles.csv     
replace users_interactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: users_interactions.csv  


In [20]:
import pandas as pd 
import numpy as np 

df1=pd.read_csv('shared_articles.csv')
df2=pd.read_csv('users_interactions.csv')

In [21]:
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [22]:
df2.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [23]:
df1=df1[df1['eventType']=='CONTENT SHARED']
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [24]:
def total_events(df1_row):
  total_likes=df2[(df2['contentId']==df1_row['contentId'])& (df2['eventType']=='LIKE')].shape[0]
  total_views=df2[(df2['contentId']==df1_row['contentId'])& (df2['eventType']=='VIEW')].shape[0]
  total_bookmarks=df2[(df2['contentId']==df1_row['contentId'])& (df2['eventType']=='BOOKMARK')].shape[0]
  total_follows=df2[(df2['contentId']==df1_row['contentId'])& (df2['eventType']=='FOLLOW')].shape[0]
  total_comments=df2[(df2['contentId']==df1_row['contentId'])& (df2['eventType']=='COMMENT CREATED')].shape[0]
  return total_likes+total_views+total_bookmarks+total_follows+total_comments
df1['total_events']=df1.apply(total_events,axis=1)
df1=df1.sort_values(['total_events'],ascending=False)
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,total_events
3096,1487246811,CONTENT SHARED,-4029704725707465084,6013226412048763966,-6569695881431984742,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,HTML,http://www.cnbc.com/2016/12/21/former-google-c...,Former Google career coach shares a visual tri...,"If you want 2017 to be an exciting year, desig...",en,433
1671,1467813728,CONTENT SHARED,-133139342397538859,4918484843075254252,-5701227433817087697,NaN,NaN,NaN,HTML,http://gq.globo.com/Prazeres/Poder/Carreira/no...,"Novo workaholic trabalha, pratica esportes e t...",Novo workaholic não abre mão do esporte e da f...,pt,315
1814,1468867647,CONTENT SHARED,-6783772548752091658,4918484843075254252,-8995217520473210153,NaN,NaN,NaN,HTML,http://www.caroli.org/livro-retrospectivas-div...,Livro: Retrospectivas Divertidas,"Neste livro, nós fornecemos um conjunto de fer...",pt,294
1317,1465484901,CONTENT SHARED,8657408509986329668,-8020832670974472349,838596071610016700,NaN,NaN,NaN,HTML,https://medium.com/practical-blend/pull-reques...,Pull request first - Practical Blend,Pull request first After two years of working ...,en,294
588,1461629452,CONTENT SHARED,-6843047699859121724,7527226129639571966,-1297230017812472163,NaN,NaN,NaN,HTML,https://medium.com/@jeffersoncn/ganhe-6-meses-...,"Ganhe 6 meses de acesso ao Pluralsight, maior ...","Ganhe 6 meses de acesso ao Pluralsight, maior ...",pt,281


In [26]:
def convert(x):
  if isinstance(x,str):
    return x.lower()
  else:
    return''
df1['title']=df1['title'].apply(convert)
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,total_events
3096,1487246811,CONTENT SHARED,-4029704725707465084,6013226412048763966,-6569695881431984742,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,HTML,http://www.cnbc.com/2016/12/21/former-google-c...,former google career coach shares a visual tri...,"If you want 2017 to be an exciting year, desig...",en,433
1671,1467813728,CONTENT SHARED,-133139342397538859,4918484843075254252,-5701227433817087697,NaN,NaN,NaN,HTML,http://gq.globo.com/Prazeres/Poder/Carreira/no...,"novo workaholic trabalha, pratica esportes e t...",Novo workaholic não abre mão do esporte e da f...,pt,315
1814,1468867647,CONTENT SHARED,-6783772548752091658,4918484843075254252,-8995217520473210153,NaN,NaN,NaN,HTML,http://www.caroli.org/livro-retrospectivas-div...,livro: retrospectivas divertidas,"Neste livro, nós fornecemos um conjunto de fer...",pt,294
1317,1465484901,CONTENT SHARED,8657408509986329668,-8020832670974472349,838596071610016700,NaN,NaN,NaN,HTML,https://medium.com/practical-blend/pull-reques...,pull request first - practical blend,Pull request first After two years of working ...,en,294
588,1461629452,CONTENT SHARED,-6843047699859121724,7527226129639571966,-1297230017812472163,NaN,NaN,NaN,HTML,https://medium.com/@jeffersoncn/ganhe-6-meses-...,"ganhe 6 meses de acesso ao pluralsight, maior ...","Ganhe 6 meses de acesso ao Pluralsight, maior ...",pt,281


In [29]:
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer(stop_words='english')
count_matrix=count.fit_transform(df1['title'])
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2=cosine_similarity(count_matrix,count_matrix)
df1=df1.reset_index()
indices=pd.Series(df1.index,index=df1['contentId'])

In [31]:
def get_recommedations(contentId,cosine_sim):
  idx=indices[contentId]
  sim_scores=list(enumerate(cosine_sim[idx]))
  sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
  sim_scores=sim_scores[1:11]
  movie_indices=[i[0]for i in sim_scores]
  return df2['contentId'].iloc[movie_indices]
get_recommedations(-4029704725707465084,cosine_sim2)

817     6850500272809381909
698    -7463305179076477879
86      -692972306229904743
181    -5997769775112032630
209     5964888566269323942
1109    8657408509986329668
1284    8657408509986329668
1862   -9192549002213406534
2632   -1981734999963962468
319     7429409096643373334
Name: contentId, dtype: int64